# Imports

In [ ]:
import pandas as pd
import tensorflow as tf

from Data import Data
from TrainingSection import ClassificationTrainingSection
from TestSection import ClassificationTestSection
from FramewiseModels.VGG import VGG
from Visualizer import GradCAM
from Annotation import SequentialAnnotation

# Setup

In [ ]:
# Path to config file.
config_file = 'config.ini'

# Load dataset.
data = Data(config_file=config_file)
data.load(dataset='aie')

# Custom annotation

In [ ]:
sa = SequentialAnnotation(config_file=config_file,
                         dataset='aie',
                         file_name='testing',
                         inter_sequence_skip=12,
                         intra_sequence_skip=1,
                         sequence_size=64)
sa.run()

# Training, validation and testing partitions

In [ ]:
hash_split = data.new_split(names=['train', 'val', 'test'], props=[0.5, 0.25, 0.25])
split = data.load_split(hash=hash_split)

# DataGenerators

In [ ]:

# Train DataGenerator.
train = data.load_data_gen(split=split,
                         partition='train',
                         category='framewise_classification',
                         annotation='video_level_annotation',
                         classes=['assault', 'non-assault'],
                         modes=['frame_rgb'],
                         inter_sequence_skip=10,
                         target_size=(224, 224),
                         batch_size=8)

# Validation DataGenerator.
val = data.load_data_gen(split=split,
                         partition='val',
                         category='framewise_classification',
                         annotation='video_level_annotation',
                         classes=['assault', 'non-assault'],
                         modes=['frame_rgb'],
                         inter_sequence_skip=10,
                         target_size=(224, 224),
                         batch_size=8)

# Test DataGenerator.
test = data.load_data_gen(split=split,
                         partition='test',
                         category='framewise_classification',
                         annotation='video_level_annotation',
                         classes=['assault', 'non-assault'],
                         modes=['frame_rgb'],
                         inter_sequence_skip=10,
                         target_size=(224, 224),
                         batch_size=8)

# Train

In [ ]:
# Model architeture.
model = VGG(input_shape=(224, 224, 3), n_classes=2)

# Train model.
exp = ClassificationTrainingSection(config_file=config_file)
exp.set_train_data(train)
exp.set_validation_data(val)
exp.set_model(model)

exp.run(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        train_acc=tf.keras.metrics.CategoricalAccuracy(),
        validation_acc=tf.keras.metrics.CategoricalAccuracy(),
        epochs=400)

# Test

In [ ]:
ts = ClassificationTestSection(config_file=config_file)
ts.set_testing_data(test)
ts.set_model(hash='84670223739194194574851698913430905233') # Model hash.
ts.run(name='all',
       loss=tf.keras.losses.CategoricalCrossentropy(),
       test_acc=tf.keras.metrics.CategoricalAccuracy())

ts.accuracy_by_vote('framewise_vgg_84670223739194194574851698913430905233_all.json')

# Grad-CAM

In [ ]:
gd = GradCAM(config_file=config_file)
gd.set_model(hash='84670223739194194574851698913430905233') # Model hash.
gd.set_layer_name('conv2d_14')
gd.show_sample(datagen=test, name='fi471_xvid.avi', frame_idx=3)